Import necessities

In [1]:
import tensorflow as tf
from keras.layers import Dense, Conv2DTranspose, LeakyReLU, Reshape, BatchNormalization, Activation, Conv2D, Flatten, Dropout, Input, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#Use google colab gpu
tf.test.gpu_device_name()


'/device:GPU:0'

In [2]:
#connect google drive
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Humans

/content/gdrive/My Drive/Humans


In [ ]:
# !tar -zxvf lfw.tgz #ONLY RUN ONCE

In [4]:
%cd lfw

/content/gdrive/My Drive/Humans/lfw


In [5]:
import os
from keras.preprocessing.image import ImageDataGenerator




# Create an ImageDataGenerator for preprocessing and data loading
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
)

# Create a generator for loading and preprocessing the images
batch_size = 8
image_generator = datagen.flow_from_directory(
    os.getcwd(),
    target_size=(250, 250),  # Adjust the size as needed
    batch_size=batch_size,
    class_mode=None,  # Set to None for GAN training
    shuffle=True  # Shuffle the dataset
)


Found 13233 images belonging to 5749 classes.


In [6]:

# Generator model
generator = Sequential([
  Dense(250 * 250 * 3, activation='sigmoid', input_shape=(100,)),
Reshape((250, 250, 3))
])
generator.summary()

generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 187500)            18937500  
                                                                 
 reshape (Reshape)           (None, 250, 250, 3)       0         
                                                                 
Total params: 18937500 (72.24 MB)
Trainable params: 18937500 (72.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Discriminator model
discriminator = Sequential([
    Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=(250, 250, 3), activation='relu'),
    Conv2D(128, (4, 4), strides=(2, 2), padding='same', activation='relu'),
    Flatten(),
    Dense(1, activation='sigmoid')
])

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])


In [8]:
# Combine the generator and discriminator into a GAN
discriminator.trainable = False
gan_input = Input(shape=(100,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)

# Compile the GAN
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

In [ ]:
import numpy as np

num_epochs = 1  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    for batch in image_generator:
        # Train the discriminator on real images
        real_images = batch
        batch_size = real_images.shape[0]
        real_labels = np.ones((batch_size, 1))
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        # Generate fake images and train the discriminator on them
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator.predict(noise)
        batch_size = fake_images.shape[0]
        fake_labels = np.zeros((batch_size, 1))
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        print(f"Epoch {epoch}/{num_epochs}, D Loss Real: {d_loss_real}, D Loss Fake: {d_loss_fake}, G Loss: {g_loss}")





1/1 [==============================] - 0s 34ms/step
Epoch 0/1, D Loss Real: [4.599064240551343e-09, 1.0], D Loss Fake: [0.12946529686450958, 1.0], G Loss: 2.2134346961975098
1/1 [==============================] - 0s 28ms/step
Epoch 0/1, D Loss Real: [1.5737998305028315e-18, 1.0], D Loss Fake: [0.10914848744869232, 1.0], G Loss: 2.3890786170959473
1/1 [==============================] - 0s 32ms/step
Epoch 0/1, D Loss Real: [0.20201262831687927, 0.875], D Loss Fake: [0.0850176066160202, 1.0], G Loss: 2.531273126602173
1/1 [==============================] - 0s 28ms/step
Epoch 0/1, D Loss Real: [1.1363121271133423, 0.75], D Loss Fake: [0.15476810932159424, 1.0], G Loss: 1.8994313478469849
1/1 [==============================] - 0s 25ms/step
Epoch 0/1, D Loss Real: [0.3562886118888855, 0.75], D Loss Fake: [0.22245009243488312, 1.0], G Loss: 1.6632897853851318
1/1 [==============================] - 0s 27ms/step
Epoch 0/1, D Loss Real: [0.23205186426639557, 0.875], D Loss Fake: [0.2541841268539

In [15]:
#export trained model


gan.save("deepfake_detector.h5")

TypeError: ignored

In [10]:
#Download trained model
from google.colab import files

files.download('deepfake_detector.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>